In [1]:
using BenchmarkTools
using Combinatorics

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("../phase2/comp_connexes.jl")
include("../phase1/main.jl")
include("../phase2/queue.jl")
include("../phase2/heuristics.jl")
include("../phase2/Kruskal.jl")
include("../phase2/prims_algorithm.jl")

prims_algorithm

In [2]:
tree_root = TreeNode("1", 0)
child1 = TreeNode("2", 0)
child2 = TreeNode("3", 0)
child3 = TreeNode("4", 0)
child4 = TreeNode("5",0)
child5 = TreeNode("6",0)
child6 = TreeNode("7",0)

tree = Tree("test tree", Vector{TreeNode{Int}}())

first_row = [ child1, child2, child3]
second_row = [child4, child6]
add_node!(tree, tree_root)
for child in first_row
    add_node!(tree, child, parent = tree_root)
end
for child in second_row
    add_node!(tree, child, parent = child3)
end
add_node!(tree, child5, parent = child4)
show(tree)

Node 1 has  rank 0 and 3 children.
It has no parent.
Its children are: 
     2
     3
     4
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 4 has  rank 0 and 2 children.
Its parent is 1
Its children are: 
     5
     7
Node 5 has  rank 0 and 1 children.
Its parent is 4
Its children are: 
     6
Node 7 has  rank 0 and 0 children.
Its parent is 4
It has no children.
Node 6 has  rank 0 and 0 children.
Its parent is 5
It has no children.


In [3]:
find_root(tree)

TreeNode{Int64}("1", 0, nothing, [2, 3, 4], 0, 1)

In [4]:
children_loc(child4)

1-element Vector{Int64}:
 7

In [5]:
println(tree_root)

TreeNode{Int64}("1", 0, nothing, [2, 3, 4], 0, 1)


In [6]:
n1= TreeNode("1", 0)
n2 = TreeNode("2", 0)
n3 = TreeNode("3", 0)
n4= TreeNode("4", 0)
t1 = Tree("test tree" , Vector{TreeNode{Int}}())
t1 = add_node!(t1, n1)
t1 = add_node!(t1, n2, parent = n1)
t1 = add_node!(t1, n3)
t1 = add_node!(t1, n4, parent = n3)
rank_union!(t1, n1, n3)


TreeNode{Int64}("1", 0, nothing, [2, 3], 1, 1)

In [7]:
show(t1)

Node 1 has  rank 1 and 2 children.
It has no parent.
Its children are: 
     2
     3
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 1 children.
Its parent is 1
Its children are: 
     4
Node 4 has  rank 0 and 0 children.
Its parent is 3
It has no children.


In [8]:
"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph) 
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (i, node) in enumerate(nodes(graph))
        adj_dict[i] = Dict()
        correspondance_dict[name(node)] = i
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        idx1 = correspondance_dict[name(node1)]
        idx2 = correspondance_dict[name(node2)]
        adj_dict[idx1][idx2] = weight(edge)
        adj_dict[idx2][idx1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{TreeNode{Y}}}()
    tree = Tree(name(graph), Vector{TreeNode{Y}}())
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_node)
        else
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_node)
        end
        add_node!(tree, blank_node)
        push!(priority_queue, priority_item)
    end
    return priority_queue, tree
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue, tree = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor_idx, edge_weight) in adjacency_list[index(data(priority_node))]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                #println("nodes(tree)[neighbor_idx] = ", nodes(tree)[neighbor_idx])
                if index(data(item)) == neighbor_idx
                    if  edge_weight < priority(item)
                        change_parent!(tree, nodes(tree)[neighbor_idx], data(priority_node))
                        change_rank!( nodes(tree)[neighbor_idx], edge_weight)
                        priority!(item, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return tree, data(root)
end

prims_algorithm

In [9]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end


In [10]:
priority_queue, tree = prims_priority_queue(graphe_test, "a")

#@benchmark p_tree, root = prims_algorithm(graphe_test, start_node_name = "a")

(PriorityQueue{PriorityItem{TreeNode{Vector{Float64}}}}(PriorityItem{TreeNode{Vector{Float64}}}[PriorityItem{TreeNode{Vector{Float64}}}(0, TreeNode{Vector{Float64}}("a", [0.0], nothing, Int64[], 0, 1)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("b", [0.0], nothing, Int64[], 0, 2)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("c", [0.0], nothing, Int64[], 0, 3)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("d", [0.0], nothing, Int64[], 0, 4)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("e", [0.0], nothing, Int64[], 0, 5)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("f", [0.0], nothing, Int64[], 0, 6)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("g", [0.0], nothing, Int64[], 0, 7)), PriorityItem{TreeNode{Vector{Float64}}}(Inf, TreeNode{Vector{Float64}}("h", [0.0], nothing, Int64[], 0, 8)), PriorityItem{TreeNode{Vector{Float

In [11]:
#@benchmark kruskal(graphe_test)

In [12]:
@time p_tree1, root1 = prims_algorithm(graphe_test, start_node_name = "a")

  0.170905 seconds (405.87 k allocations: 27.094 MiB, 99.78% compilation time)


(Tree{Vector{Float64}}("Test", TreeNode{Vector{Float64}}[TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1), TreeNode{Vector{Float64}}("b", [0.0], 1, [3], 4, 2), TreeNode{Vector{Float64}}("c", [0.0], 2, [4, 6, 9], 8, 3), TreeNode{Vector{Float64}}("d", [0.0], 3, [5], 7, 4), TreeNode{Vector{Float64}}("e", [0.0], 4, Int64[], 9, 5), TreeNode{Vector{Float64}}("f", [0.0], 3, [7], 4, 6), TreeNode{Vector{Float64}}("g", [0.0], 6, [8], 2, 7), TreeNode{Vector{Float64}}("h", [0.0], 7, Int64[], 1, 8), TreeNode{Vector{Float64}}("i", [0.0], 3, Int64[], 2, 9)]), TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1))

In [13]:
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
println("running Prim's algorithm on swiss42")
@time swiss_42_tree, root2 = prims_algorithm(swiss_42_graph)
swiss_42_graph = tree_to_graph(swiss_42_tree, root2)
sum_of_weights(swiss_42_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm on swiss42
  0.015656 seconds (17.91 k allocations: 1013.049 KiB, 93.25% compilation time)


1079.0

In [14]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree, root3 = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree, root3)

sum_of_weights(bays_29_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm
  0.000477 seconds (4.43 k allocations: 214.359 KiB)


1557.0

## Algorithm de Held et Karp 

In [15]:
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee]
#Edges are fully connected
edge1 = Edge(nodea,nodeb, 4.)
edge2 = Edge(nodea,nodec, 8.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 8.)
edge5 = Edge(nodeb, nodec, 7.)
edge6 = Edge(nodeb, noded, 1.)
edge7 = Edge(nodeb, nodee, 6.)
edge8 = Edge(nodec, noded, 2.)
edge9 = Edge(nodec, nodee, 4.)
edge10 = Edge(noded, nodee, 7.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10]

tsp_test = Graph("Test",node_list,edge_list)
tsp_test


Graph{Vector{Float64}, Float64}("Test", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 4.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("d", [0.0]), 11.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("e", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("d", [0.0]), 1.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}(

In [16]:
#Fully connected graph with 7 edges
a = Node("a",[0.])
b = Node("b",[0.])
c = Node("c",[0.])
d = Node("d",[0.])
e = Node("e",[0.])
f = Node("f",[0.])
g = Node("g",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, nodeg]
#Edges are fully connected
e1 = Edge(nodea,nodeb, 4.)
edge2 = Edge(nodea,nodec, 8.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 8.)
edge5 = Edge(nodea, nodef, 7.)
edge6 = Edge(nodea, nodeg, 1.)
edge7 = Edge(nodeb, nodec, 6.)
edge8 = Edge(nodeb, noded, 2.)
edge9 = Edge(nodeb, nodee, 4.)
edge10 = Edge(nodeb, nodef, 7.)
edge11 = Edge(nodeb, nodeg, 2.)
edge12 = Edge(nodec, noded, 7.)
edge13 = Edge(nodec, nodee, 1.)
edge14 = Edge(nodec, nodef, 6.)
edge15 = Edge(nodec, nodeg, 3.)
edge16 = Edge(noded, nodee, 5.)
edge17 = Edge(noded, nodef, 4.)
edge18 = Edge(noded, nodeg, 8.)
edge19 = Edge(nodee, nodef, 2.)
edge20 = Edge(nodee, nodeg, 7.)
edge21 = Edge(nodef, nodeg, 3.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14, edge15, edge16,
                edge17, edge18, edge19, edge20, edge21]
#creates the graph
tsp_test2 = Graph("Test2",node_list,edge_list)

Graph{Vector{Float64}, Float64}("Test2", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0]), Node{Vector{Float64}}("f", [0.0]), Node{Vector{Float64}}("g", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 4.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("d", [0.0]), 11.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("e", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("f", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("g", [0.0]), 1.0), Edge{Float64, Vec

In [17]:
function h_k_exact_algorithm(g::Graph ; start_node_name::Any = nothing)
    #if no start node name, first node is start node
    if isnothing(start_node_name)
        start_node_name = name(nodes(g)[1])
    end
    #move start node to first position of graph
    for (i, node) in enumerate(nodes(g))
        if name(node) == start_node_name
            g.nodes[1], g.nodes[i] = g.nodes[i], g.nodes[1]
            break
        end
    end
    #create adjacency list
    adjacency_list = adjacency_dict(g)
    distance_dict = Dict()
    for k in 2:length(nodes(g))
        distance_dict[(Set([k]), k)] = Dict("c" =>adjacency_list[1][k], "p"=> [1, k])
    end
    for s in 2:length(nodes(g))-1
        for subset in combinations(2:length(nodes(g)), s)
            for k in subset
                min_dist = Inf
                for m in subset 
                    if m !=k
                        predecessor = distance_dict[(setdiff(Set(subset), Set([k])), m)]
                        #println("predecessor", predecessor)
                        dist = predecessor["c"] + adjacency_list[m][k]
                        if dist < min_dist
                            min_dist = dist
                            min_m = m
                            new_list = copy(predecessor["p"])
                            append!(new_list, k)
                            distance_dict[(Set(subset), k)] = Dict("c" => min_dist, "p" => new_list)
                        end
                    end
                end
            end
        end
    end

    #returns the minimum path from the start node that goes to all of the other paths
    min = Inf
    final_path = []
    for k in 2:length(nodes(g))
        distance = distance_dict[(Set(2:length(nodes(g))), k)]["c"] + adjacency_list[k][1]
        println("distance = ", distance)
        if distance < min
            println("distance = ", distance)
            min = distance
            final_path = copy(distance_dict[(Set(2:length(nodes(g))), k)]["p"])
            append!(final_path, 1)
        end
    end
    return min, final_path
end


h_k_algorithm(tsp_test)


UndefVarError: UndefVarError: `h_k_algorithm` not defined

In [39]:


function get_shortest_edges(graph::Graph, departure_node::Node)
    shortest_edge_vec = Vector{Edge}(undef, 2)
    shortest_dist_vec = Vector{Float64}([Inf, Inf])
    for  edge in edges(graph)
        node1, node2 = nodes(edge)
        if name(node1) == name(departure_node) || name(node2) == name(departure_node)
            #copies over the previous best to second best
            if weight(edge) < shortest_dist_vec[1]
                if shortest_dist_vec[1]< shortest_dist_vec[2]
                    shortest_dist_vec[2] = shortest_dist_vec[1]
                    shortest_edge_vec[2] = shortest_edge_vec[1]
                end
                shortest_dist_vec[1] = weight(edge)
                shortest_edge_vec[1] = edge
            elseif weight(edge) < shortest_dist_vec[2]
                shortest_dist_vec[2] = weight(edge)
                shortest_edge_vec[2] = edge
            end
        end
    end
    return  shortest_edge_vec[1], shortest_edge_vec[2]
end



function find_one_tree(graph::Graph, departure_node::Node, adjacency_list::Dict)
    #Copies graph to feed into prim's algorithm
    start_graph = deepcopy(graph)
    #removes the departure node from the graph and saves its index
    idx = remove_node!(start_graph, departure_node)
    remove_edges!(start_graph, departure_node)
    #uses prims algorithm to find the one_tree
    one_tree, root = prims_algorithm(start_graph, start_node_name = name(departure_node))
    #converts the prims algorithm to a graph
    one_tree_graph = tree_to_graph(one_tree, root)
    #finds the second shortest edge from the departure node
    shortest_edge_1, shortest_edge_2   = get_shortest_edges(graph, departure_node)
    #prints the graph if one of the edges are nothing
    if isnothing(shortest_edge_1) || isnothing(shortest_edge_2)
        println("nothing edge")
        println("shortest edge1", shortest_edge_1)
        println("shortest edge2", shortest_edge_2)
        println("one_tree_graph = ", )
        show(one_tree_graph)
        println("departure_node = ", departure_node)
        println("idx = ", idx)
        println("graph")
        show(graph)
    end
    add_node!(one_tree_graph, departure_node, idx)
    add_edge!(one_tree_graph,shortest_edge_1)
    add_edge!(one_tree_graph,shortest_edge_2)
    one_tree_distance = sum_of_weights(one_tree_graph)
    return one_tree_distance, one_tree_graph
end

#Returns the degrees of nodes in  a graph
function degree(graph::Graph)
    adjacency_list = adjacency_dict(graph)
    degrees = []
    for i in 1:length(nodes(graph))
        push!(degrees, length(keys(adjacency_list[i])))
    end
    return degrees
end

function update_edge_weights!(graph::Graph, pis::Vector{Float64})
    correspondance_dict = Dict()
    for (i, node) in enumerate(nodes(graph))
        correspondance_dict[name(node)] = i
    end
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        node1_idx = correspondance_dict[name(node1)]
        node2_idx = correspondance_dict[name(node2)]
        new_weight = weight(edge) + pis[node1_idx] + pis[node2_idx]
        set_weight!(edge, new_weight)
    end
end

"""subgradient heuristic for calculating a minimal tour"""
function lkh_subgradient(start_graph::Graph;  departure_node::Union{Node, Nothing} = nothing)
    if isnothing(departure_node)
        departure_node = nodes(start_graph)[1]
    end
    graph = deepcopy(start_graph)
    k = 0
    w = -Inf
    pis = zeros(length(nodes(graph)))
    adjacency_list = adjacency_dict(graph)
    while true && k < 100000
        total_distance, one_tree = find_one_tree(graph, departure_node, adjacency_list)
        weights_k = total_distance - 2 * sum(pis)
        w = max(w, weights_k)
        v_k = degree(one_tree) .- 2
        println("k ", k)
        if v_k == zeros(length(nodes(graph)))
            return total_distance, one_tree
        end
        t_k = 200/ log(k+2)
        pis = pis + t_k * v_k
        k = k + 1
        update_edge_weights!(graph, pis)
    end
    return Inf, nothing
end

lkh_subgradient

In [40]:
lkh_subgradient(tsp_test, departure_node = nodea)

k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183
k 184


271
k 272
k 273
k 274
k 275
k 276
k 277
k 278
k 279
k 280
k 281
k 282
k 283
k 284
k 285
k 286
k 287
k 288
k 289
k 290
k 291
k 292
k 293
k 294
k 295
k 296
k 297
k 298
k 299
k 300
k 301
k 302
k 303
k 304
k 305
k 306
k 307
k 308
k 309
k 310
k 311
k 312
k 313
k 314
k 315
k 316
k 317
k 318
k 319
k 320
k 321
k 322
k 323
k 324
k 325
k 326
k 327
k 328
k 329
k 330
k 331
k 332
k 333
k 334
k 335
k 336
k 337
k 338
k 339
k 340
k 341
k 342
k 343
k 344
k 345
k 346
k 347
k 348
k 349
k 350
k 351
k 352
k 353
k 354
k 355
k 356
k 357
k 358
k 359
k 360
k 361
k 362
k 363
k 364
k 365
k 366
k 367
k 368
k 369
k 370
k 371
k 372
k 373
k 374
k 375
k 376
k 377
k 378
k 379
k 380
k 381
k 382
k 383
k 384
k 385
k 386
k 387
k 388
k 389
k 390
k 391
k 392
k 393
k 394
k 395
k 396
k 397
k 398
k 399
k 400
k 401
k 402
k 403
k 404
k 405
k 406
k 407
k 408
k 409
k 410
k 411
k 412
k 413
k 414
k 415
k 416
k 417
k 418
k 419
k 420
k 421
k 422
k 423
k 424
k 425
k 426
k 427
k 428
k 429
k 430
k 431
k 432
k 433
k 434
k 435
k 436
k 437


1359
k 1360
k 1361
k 1362
k 1363
k 1364
k 1365
k 1366
k 1367
k 1368
k 1369
k 1370
k 1371
k 1372
k 1373
k 1374
k 1375
k 1376
k 1377
k 1378
k 1379
k 1380
k 1381
k 1382
k 1383
k 1384
k 1385
k 1386
k 1387
k 1388
k 1389
k 1390
k 1391
k 1392
k 1393
k 1394
k 1395
k 1396
k 1397
k 1398
k 1399
k 1400
k 1401
k 1402
k 1403
k 1404
k 1405
k 1406
k 1407
k 1408
k 1409
k 1410
k 1411
k 1412
k 1413
k 1414
k 1415
k 1416
k 1417
k 1418
k 1419
k 1420
k 1421
k 1422
k 1423
k 1424
k 1425
k 1426
k 1427
k 1428
k 1429
k 1430
k 1431
k 1432
k 1433
k 1434
k 1435
k 1436
k 1437
k 1438
k 1439
k 1440
k 1441
k 1442
k 1443
k 1444
k 1445
k 1446
k 1447
k 1448
k 1449
k 1450
k 1451
k 1452
k 1453
k 1454
k 1455
k 1456
k 1457
k 1458
k 1459
k 1460
k 1461
k 1462
k 1463
k 1464
k 1465
k 1466
k 1467
k 1468
k 1469
k 1470
k 1471
k 1472
k 1473
k 1474
k 1475
k 1476
k 1477
k 1478
k 1479
k 1480
k 1481
k 1482
k 1483
k 1484
k 1485
k 1486
k 1487
k 1488
k 1489
k 1490
k 1491
k 1492
k 1493
k 1494
k 1495
k 1496
k 1497
k 1498
k 1499
k 1500
k 1501
k


k 2577
k 2578
k 2579
k 2580
k 2581
k 2582
k 2583
k 2584
k 2585
k 2586
k 2587
k 2588
k 2589
k 2590
k 2591
k 2592
k 2593
k 2594
k 2595
k 2596
k 2597
k 2598
k 2599
k 2600
k 2601
k 2602
k 2603
k 2604
k 2605
k 2606
k 2607
k 2608
k 2609
k 2610
k 2611
k 2612
k 2613
k 2614
k 2615
k 2616
k 2617
k 2618
k 2619
k 2620
k 2621
k 2622
k 2623
k 2624
k 2625
k 2626
k 2627
k 2628
k 2629
k 2630
k 2631
k 2632
k 2633
k 2634
k 2635
k 2636
k 2637
k 2638
k 2639
k 2640
k 2641
k 2642
k 2643
k 2644
k 2645
k 2646
k 2647
k 2648
k 2649
k 2650
k 2651
k 2652
k 2653
k 2654
k 2655
k 2656
k 2657
k 2658
k 2659
k 2660
k 2661
k 2662
k 2663
k 2664
k 2665
k 2666
k 2667
k 2668
k 2669
k 2670
k 2671
k 2672
k 2673
k 2674
k 2675
k 2676
k 2677
k 2678
k 2679
k 2680
k 2681
k 2682
k 2683
k 2684
k 2685
k 2686
k 2687
k 2688
k 2689
k 2690
k 2691
k 2692
k 2693
k 2694
k 2695
k 2696
k 2697
k 2698
k 2699
k 2700
k 2701
k 2702
k 2703
k 2704
k 2705
k 2706
k 2707
k 2708
k 2709
k 2710
k 2711
k 2712
k 2713
k 2714
k 2715
k 2716
k 2717
k 2718
k 271

3741
k 3742
k 3743
k 3744
k 3745
k 3746
k 3747
k 3748
k 3749
k 3750
k 3751
k 3752
k 3753
k 3754
k 3755
k 3756
k 3757
k 3758
k 3759
k 3760
k 3761
k 3762
k 3763
k 3764
k 3765
k 3766
k 3767
k 3768
k 3769
k 3770
k 3771
k 3772
k 3773
k 3774
k 3775
k 3776
k 3777
k 3778
k 3779
k 3780
k 3781
k 3782
k 3783
k 3784
k 3785
k 3786
k 3787
k 3788
k 3789
k 3790
k 3791
k 3792
k 3793
k 3794
k 3795
k 3796
k 3797
k 3798
k 3799
k 3800
k 3801
k 3802
k 3803
k 3804
k 3805
k 3806
k 3807
k 3808
k 3809
k 3810
k 3811
k 3812
k 3813
k 3814
k 3815
k 3816
k 3817
k 3818
k 3819
k 3820
k 3821
k 3822
k 3823
k 3824
k 3825
k 3826
k 3827
k 3828
k 3829
k 3830
k 3831
k 3832
k 3833
k 3834
k 3835
k 3836
k 3837
k 3838
k 3839
k 3840
k 3841
k 3842
k 3843
k 3844
k 3845
k 3846
k 3847
k 3848
k 3849
k 3850
k 3851
k 3852
k 3853
k 3854
k 3855
k 3856
k 3857
k 3858
k 3859
k 3860
k 3861
k 3862
k 3863
k 3864
k 3865
k 3866
k 3867
k 3868
k 3869
k 3870
k 3871
k 3872
k 3873
k 3874
k 3875
k 3876
k 3877
k 3878
k 3879
k 3880
k 3881
k 3882
k 3883
k

...

k 60838
k 60839
k 60840
k 60841
k 60842
k 60843
k 60844
k 60845
k 60846
k 60847
k 60848
k 60849
k 60850
k 60851
k 60852
k 60853
k 60854
k 60855
k 60856
k 60857
k 60858
k 60859
k 60860
k 60861
k 60862
k 60863
k 60864
k 60865
k 60866
k 60867
k 60868
k 60869
k 60870
k 60871
k 60872
k 60873
k 60874
k 60875
k 60876
k 60877
k 60878
k 60879
k 60880
k 60881
k 60882
k 60883
k 60884
k 60885
k 60886
k 60887
k 60888
k 60889
k 60890
k 60891
k 60892
k 60893
k 60894
k 60895
k 60896
k 60897
k 60898
k 60899
k 60900
k 60901
k 60902
k 60903
k 60904
k 60905
k 60906
k 60907
k 60908
k 60909
k 60910
k 60911
k 60912
k 60913
k 60914
k 60915
k 60916
k 60917
k 60918
k 60919
k 60920
k 60921
k 60922
k 60923
k 60924
k 60925
k 60926
k 60927
k 60928
k 60929
k 60930
k 60931
k 60932
k 60933
k 60934
k 60935
k 60936
k 60937
k 60938
k 60939
k 60940
k 60941
k 60942
k 60943
k 60944
k 60945
k 60946
k 60947
k 60948
k 60949
k 60950
k 60951
k 60952
k 60953
k 60954
k 60955
k 60956
k 60957
k 60958
k 60959
k 60960
k 60961
k 60962


61749
k 61750
k 61751
k 61752
k 61753
k 61754
k 61755
k 61756
k 61757
k 61758
k 61759
k 61760
k 61761
k 61762
k 61763
k 61764
k 61765
k 61766
k 61767
k 61768
k 61769
k 61770
k 61771
k 61772
k 61773
k 61774
k 61775
k 61776
k 61777
k 61778
k 61779
k 61780
k 61781
k 61782
k 61783
k 61784
k 61785
k 61786
k 61787
k 61788
k 61789
k 61790
k 61791
k 61792
k 61793
k 61794
k 61795
k 61796
k 61797
k 61798
k 61799
k 61800
k 61801
k 61802
k 61803
k 61804
k 61805
k 61806
k 61807
k 61808
k 61809
k 61810
k 61811
k 61812
k 61813
k 61814
k 61815
k 61816
k 61817
k 61818
k 61819
k 61820
k 61821
k 61822
k 61823
k 61824
k 61825
k 61826
k 61827
k 61828
k 61829
k 61830
k 61831
k 61832
k 61833
k 61834
k 61835
k 61836
k 61837
k 61838
k 61839
k 61840
k 61841
k 61842
k 61843
k 61844
k 61845
k 61846
k 61847
k 61848
k 61849
k 61850
k 61851
k 61852
k 61853
k 61854
k 61855
k 61856
k 61857
k 61858
k 61859
k 61860
k 61861
k 61862
k 61863
k 61864
k 61865
k 61866
k 61867
k 61868
k 61869
k 61870
k 61871
k 61872
k 61873
k 

Excessive output truncated after 524292 bytes.


k 63005
k 63006
k 63007
k 63008
k 63009
k 63010
k 63011
k 63012
k 63013
k 63014
k 63015
k 63016
k 63017
k 63018
k 63019
k 63020
k 63021
k 63022
k 63023
k 63024
k 63025
k 63026
k 63027
k 63028
k 63029
k 63030
k 63031
k 63032
k 63033
k 63034
k 63035
k 63036
k 63037
k 63038
k 63039
k 63040
k 63041
k 63042
k 63043
k 63044
k 63045
k 63046
k 63047
k 63048
k 63049
k 63050
k 63051
k 63052
k 63053
k 63054
k 63055
k 63056
k 63057
k 63058
k 63059
k 63060
k 63061
k 63062
k 63063
k 63064
k 63065
k 63066
k 63067
k 63068
k 63069
k 63070
k 63071
k 63072
k 63073
k 63074
k 63075
k 63076
k 63077
k 63078
k 63079
k 63080
k 63081
k 63082
k 63083
k 63084
k 63085
k 63086
k 63087
k 63088
k 63089
k 63090
k 63091
k 63092
k 63093
k 63094
k 63095
k 63096
k 63097
k 63098
k 63099
k 63100
k 63101
k 63102
k 63103
k 63104
k 63105
k 63106
k 63107
k 63108
k 63109
k 63110
k 63111
k 63112
k 63113
k 63114
k 63115
k 63116
k 63117
k 63118
k 63119
k 63120
k 63121
k 63122
k 63123
k 63124
k 63125
k 63126
k 63127
k 63128
k 63129

(Inf, nothing)

In [20]:
show(tsp_test2)

Graph Test2 has 7 nodes.
Node a, data: 

[0.0]
Node b, data: [0.0]
Node c, data: [0.0]
Node d, data: [0.0]
Node e, data: [0.0]
Node f, data: [0.0]
Node g, data: [0.0]
Graph Test2 has 21 edges.
Arête de a à b, poids: 4.0
Arête de a à c, poids: 8.0
Arête de a à d, poids: 11.0
Arête de a à e, poids: 8.0
Arête de a à f, poids: 7.0
Arête de a à g, poids: 1.0
Arête de b à c, poids: 6.0
Arête de b à d, poids: 2.0
Arête de b à e, poids: 4.0
Arête de b à f, poids: 7.0
Arête de b à g, poids: 2.0
Arête de c à d, poids: 7.0
Arête de c à e, poids: 1.0
Arête de c à f, poids: 6.0
Arête de c à g, poids: 3.0
Arête de d à e, poids: 5.0
Arête de d à f, poids: 4.0
Arête de d à g, poids: 8.0
Arête de e à f, poids: 2.0
Arête de e à g, poids: 7.0
Arête de f à g, poids: 3.0


In [28]:
score, test2_graph = lkh_subgradient(tsp_test2)
score

k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183
k 184


Inf

In [22]:
gr17_graph, gr17_nodes = graph_from_tsp("../../instances/stsp/gr17.tsp","graphe1")
println("running h_k_algorithm on gr17")
@time total_distance, one_tree = lkh_subgradient(gr17_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running h_k_algorithm on gr17
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 

32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183
k 184
k 185
k 186
k 187
k 188
k 189
k 190
k 191
k 192
k 193
k 194
k 195
k 196
k 197
k 198
k 199
k 200
k 201
k 202
k 203
k 204
k 205
k 206
k 207
k 208
k 209
k 

280
k 281
k 282
k 283
k 284
k 285
k 286
k 287
k 288
k 289
k 290
k 291
k 292
k 293
k 294
k 295
k 296
k 297
k 298
k 299
k 300
k 301
k 302
k 303
k 304
k 305
k 306
k 307
k 308
k 309
k 310
k 311
k 312
k 313
k 314
k 315
k 316
k 317
k 318
k 319
k 320
k 321
k 322
k 323
k 324
k 325
k 326
k 327
k 328
k 329
k 330
k 331
k 332
k 333
k 334
k 335
k 336
k 337
k 338
k 339
k 340
k 341
k 342
k 343
k 344
k 345
k 346
k 347
k 348
k 349
k 350
k 351
k 352
k 353
k 354
k 355
k 356
k 357
k 358
k 359
k 360
k 361
k 362
k 363
k 364
k 365
k 366
k 367
k 368
k 369
k 370
k 371
k 372
k 373
k 374
k 375
k 376
k 377
k 378
k 379
k 380
k 381
k 382
k 383
k 384
k 385
k 386
k 387
k 388
k 389
k 390
k 391
k 392
k 393
k 394
k 395
k 396
k 397
k 398
k 399
k 400
k 401
k 402
k 403
k 404
k 405
k 406
k 407
k 408
k 409
k 410
k 411
k 412
k 413
k 414
k 415
k 416
k 417
k 418
k 419
k 420
k 421
k 422
k 423
k 424
k 425
k 426
k 427
k 428
k 429
k 430
k 431
k 432
k 433
k 434
k 435
k 436
k 437
k 438
k 439
k 440
k 441
k 442
k 443
k 444
k 445
k 446


(Inf, nothing)

In [23]:
"""Brute force calculates TSP solution"""
function brute_force_tsp(g::Graph, start_node::Any)
    adjacency_list = adjacency_dict(g)
    g_nodes = nodes(g)
    nodes_perm = collect(permutations(collect(1:length(g_nodes))))
    min_dist = Inf
    min_perm = []
    for perm in nodes_perm
        dist = 0
        for i in 1:length(perm)-1
            dist += adjacency_list[perm[i]][perm[i+1]]
        end
        dist += adjacency_list[perm[end]][perm[1]]
        println("permutation", perm)
        println("dist = ", dist)
        if dist < min_dist
            min_dist = dist
            min_perm = perm
        end
        
    end
    return min_dist, min_perm
end
brute_force_tsp(tsp_test, "a")

permutation

[1, 2, 3, 4, 5]
dist = 28.0
permutation[1, 2, 3, 5, 4]
dist = 33.0
permutation[1, 2, 4, 3, 5]
dist = 19.0
permutation[1, 2, 4, 5, 3]
dist = 24.0
permutation[1, 2, 5, 3, 4]
dist = 27.0
permutation[1, 2, 5, 4, 3]
dist = 27.0
permutation[1, 3, 2, 4, 5]
dist = 31.0
permutation[1, 3, 2, 5, 4]
dist = 39.0
permutation[1, 3, 4, 2, 5]
dist = 25.0
permutation[1, 3, 4, 5, 2]
dist = 27.0
permutation[1, 3, 5, 2, 4]
dist = 30.0
permutation[1, 3, 5, 4, 2]
dist = 24.0
permutation[1, 4, 2, 3, 5]
dist = 31.0
permutation[1, 4, 2, 5, 3]
dist = 30.0
permutation[1, 4, 3, 2, 5]
dist = 34.0
permutation[1, 4, 3, 5, 2]
dist = 27.0
permutation[1, 4, 5, 2, 3]
dist = 39.0
permutation[1, 4, 5, 3, 2]
dist = 33.0
permutation[1, 5, 2, 3, 4]
dist = 34.0
permutation[1, 5, 2, 4, 3]
dist = 25.0
permutation[1, 5, 3, 2, 4]
dist = 31.0
permutation[1, 5, 3, 4, 2]
dist = 19.0
permutation[1, 5, 4, 2, 3]
dist = 31.0
permutation[1, 5, 4, 3, 2]
dist = 28.0
permutation[2, 1, 3, 4, 5]
dist = 27.0
permutation[2, 1, 3, 5, 4]
dist = 24

(19.0, [1, 2, 4, 3, 5])

In [24]:
[15]

1-element Vector{Int64}:
 15

In [25]:
a = Dict()
a[Set([1,2])] = 1
a[Set([2,1])] = 2
a

Dict{Any, Any} with 1 entry:
  Set([2, 1]) => 2